In [14]:
from langchain.schema.runnable import RunnableConfig
from langchain_core.callbacks import BaseCallbackHandler
from langchain.schema.messages import HumanMessage
from langchain_ollama import ChatOllama
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops (useful in Jupyter notebooks)
nest_asyncio.apply()

# Custom callback handler for logging
class LoggingCallback(BaseCallbackHandler):
    def on_llm_start(self, *args, **kwargs):
        print("Starting LLM call...")
    
    def on_llm_end(self, *args, **kwargs):
        print("LLM call completed")
    
    def on_chain_start(self, *args, **kwargs):
        print("Starting chain execution...")

# Initialize the callback handler
callback_handler = LoggingCallback()

# Initialize the model
embedding_mode = "qwen2.5:14b"
# model = ChatOllama(model=embedding_mode, temperature=0, callbacks=[callback_handler])
model = ChatOllama(model=embedding_mode, temperature=0)

# Basic config with tags
basic_config: RunnableConfig = {
    "tags": ["customer-support", "urgent"],
    "metadata": {"user_id": "123", "session_id": "abc"}
}

# Config with callbacks
config_with_callbacks: RunnableConfig = {
    "callbacks": [callback_handler],
    "tags": ["support"],
    "metadata": {"priority": "high"}
}

# Config with timeout and retry settings
advanced_config: RunnableConfig = {
    "max_concurrency": 5,
    "run_name": "customer_query",
    "timeout": 10.0,  # timeout in seconds
    "tags": ["production"],
    "metadata": {
        "department": "sales",
        "version": "1.0"
    }
}

# Example usage with synchronous calls
def process_query(query: str, config: RunnableConfig):
    messages = [HumanMessage(content=query)]
    try:
        response = model.invoke(messages, config=config)
        return response.content
    except Exception as e:
        print(f"Error processing query: {e}")
        return None

# Example usage with async calls
async def process_queries_batch(queries: list[str], config: RunnableConfig):
    messages_list = [[HumanMessage(content=q)] for q in queries]
    try:
        responses = await model.abatch(messages_list, config=config)
        return [r.content for r in responses]
    except Exception as e:
        print(f"Error processing batch: {e}")
        return None

# Example usage: 
# Play with running with callbacks added directly to the model and/or at the invoke/abatch. 
if __name__ == "__main__":
    # Synchronous example
    query = "What's the most important city in the world?"
    result = process_query(query, config_with_callbacks)
    
    # Asynchronous batch example
    queries = ["What's the most important city in the world?", "Why is it important?"]
    async def run_batch():
        results = await process_queries_batch(queries, advanced_config)
        return results
    
    batch_results = asyncio.run(run_batch())

Starting LLM call...
LLM call completed


In [11]:
print(result)

Determining the "most important" city in the world can be subjective and depends on various criteria such as economic power, cultural influence, political significance, or technological innovation. Different cities excel in different areas:

1. **Economic Power**: Cities like New York City (USA) and London (UK) are often considered global financial centers due to their significant roles in international finance and trade.

2. **Political Influence**: Washington D.C. (USA), as the capital of one of the world's most powerful nations, plays a crucial role in global politics and diplomacy.

3. **Cultural Impact**: Cities like Paris (France) and Tokyo (Japan) are renowned for their cultural contributions to art, fashion, cuisine, and entertainment.

4. **Technological Innovation**: Silicon Valley (San Francisco Bay Area, USA) is pivotal in the tech industry, driving innovation and shaping global technology trends.

5. **Historical Significance**: Cities like Rome (Italy), Jerusalem (Israel/

In [12]:
print(batch_results)

['Determining the "most important" city in the world can be subjective and depends on various criteria such as economic power, cultural influence, political significance, or technological innovation. Different cities excel in different areas:\n\n1. **Economic Power**: Cities like New York City (USA) and London (UK) are often considered global financial centers due to their significant roles in international finance and trade.\n\n2. **Political Influence**: Washington D.C. (USA), as the capital of one of the world\'s most powerful nations, plays a crucial role in global politics and diplomacy.\n\n3. **Cultural Impact**: Cities like Paris (France) and Tokyo (Japan) are renowned for their cultural contributions to art, fashion, cuisine, and entertainment.\n\n4. **Technological Innovation**: Silicon Valley (San Francisco Bay Area, USA) is pivotal in the tech industry, driving innovation and shaping global technology trends.\n\n5. **Historical Significance**: Cities like Rome (Italy), Jerus